# Домашнее задание к лекции "Основы веб-скрапинга"

## Обязательная часть

Вам необходимо написать функцию, которая будет основана на **поиске** по сайту [habr.com](https://habr.com/ru/search/).
Функция в качестве параметра должна принимать **список** запросов для поиска (например, `['python', 'анализ данных']`) и на основе материалов, попавших в результаты поиска по **каждому** запросу, возвращать датафрейм вида:

```
<дата> - <заголовок> - <ссылка на материал>
```

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.


In [13]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def scrape_habr_articles(queries):
    base_url = 'https://habr.com'
    articles = []

    for query in queries:
        url = f'{base_url}/search/?q={query}&target_type=posts&order=relevance'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        articles_list = soup.find_all('article', class_='tm-articles-list__item')
        for article in articles_list:
            date_element = article.find('span', class_='tm-article-snippet__datetime-published')
            if date_element is not None:
                date = date_element.text.strip()
            else:
                date = ''

            title_element = article.find('h2', class_="tm-title tm-title_h2")
            if title_element is not None:
                title_link = title_element.find('a')
                if title_link is not None:
                    link_relative = title_link.get('href', '')
                    link_full = urljoin(base_url, link_relative)
                else:
                    link_full = ''
                title = title_element.text.strip()
            else:
                title = ''
                link_full = ''

            row = {'date': date, 'title': title, 'link': link_full}
            articles.append(row)

    return articles

# Пример использования функции
queries = ['python', 'анализ данных']
articles = scrape_habr_articles(queries)

# Выводим результаты
for article in articles:
    print(article)

{'date': '', 'title': 'Как я проектирую и разрабатываю расширения Python на Rust', 'link': 'https://habr.com/en/articles/767254/'}
{'date': '', 'title': 'Функциональное программирование в Python: ежедневные рецепты', 'link': 'https://habr.com/en/companies/kaspersky/articles/762788/'}
{'date': '', 'title': 'Многопоточность в Python: очевидное и невероятное', 'link': 'https://habr.com/en/articles/764420/'}
{'date': '', 'title': '10 лучших практик логирования в Python', 'link': 'https://habr.com/en/companies/ruvds/articles/766010/'}
{'date': '', 'title': 'Как работать с процессами и потоками в Python', 'link': 'https://habr.com/en/companies/simbirsoft/articles/701020/'}
{'date': '', 'title': 'Построение пайплайна обработки данных в реальном времени с использованием Python', 'link': 'https://habr.com/en/companies/otus/articles/764136/'}
{'date': '', 'title': 'Как настроить сбор статистики и автоматическое отключение пользователей WireGuard в ispmanager с помощью Python и API', 'link': 'htt

Подробно рассмотрим каждую строку кода:

1. `import requests`: Импортируем модуль `requests`, который позволяет отправлять HTTP-запросы и получать ответы от веб-серверов.

2. `from bs4 import BeautifulSoup`: Из модуля `bs4` импортируем класс `BeautifulSoup`, который используется для парсинга HTML-страниц.

3. `from urllib.parse import urljoin`: Из модуля `urllib.parse` импортируем функцию `urljoin`, которая позволяет объединить базовый URL и относительный URL.

4. `def scrape_habr_articles(queries)`: Определяем функцию `scrape_habr_articles`, которая принимает аргумент `queries`, содержащий список запросов.

5. `base_url = 'https://habr.com'`: Задаем базовый URL для сайта `Habr`.

6. `articles = []`: Создаем пустой список `articles`, в который будут добавлены статьи.

7. `for query in queries:`: Начинаем цикл `for`, который перебирает каждый запрос `query` из списка `queries`.

8. `url = f'{base_url}/search/?q={query}&target_type=posts&order=relevance'`: Строим URL для конкретного запроса `query`. В URL включается базовый URL `base_url` и параметры запроса, такие как поисковый запрос `q`, тип цели `target_type` и порядок сортировки `order`.

9. `response = requests.get(url)`: Отправляем GET-запрос по указанному URL и сохраняем полученный ответ в переменную `response`.

10. `soup = BeautifulSoup(response.text, 'html.parser')`: Создаем объект `BeautifulSoup` для парсинга HTML-кода страницы. Аргумент `response.text` содержит HTML-код полученного ответа, и `'html.parser'` указывает, что нужно использовать HTML-парсер.

11. `articles_list = soup.find_all('article', class_='tm-articles-list__item')`: Ищем все элементы `'article'` с классом `'tm-articles-list__item'` внутри объекта `soup` и сохраняем результат в переменную `articles_list`.

12. `for article in articles_list:`: Начинаем цикл `for`, который перебирает каждую статью `article` из списка `articles_list`.

13. `date_element = article.find('span', class_='tm-article-snippet__datetime-published')`: Ищем элемент `span` с классом `'tm-article-snippet__datetime-published'` внутри статьи `article` и сохраняем результат в переменную `date_element`.

14. `if date_element is not None:`: Проверяем, найден ли элемент `date_element`. Если элемент существует (не равен `None`), выполняем следующие действия. Если элемент не найден, переходим к строке 19.

15. `date = date_element.text.strip()`: Извлекаем текст из элемента `date_element` с помощью метода `text` и удаляем лишние пробелы с помощью метода `strip()`. Результат сохраняется в переменную `date`.

16. `else:`: Если элемент `date_element` не найден (равен `None`), выполняем следующие действия.

17. `date = ''`: Присваиваем переменной `date` пустую строку.

18. Аналогично строкам 13-17, переменные `title` и `link_full` извлекаются из статьи `article`. Первым шагом ищется элемент `h2` с классом `"tm-title tm-title_h2"`. Если элемент найден, выполняются следующие действия. Если элемент не найден, переменным `title` и `link_full` присваиваются пустые строки.

19. `row = {'date': date, 'title': title, 'link': link_full}`: Создаем словарь `row`, содержащий информацию о дате, заголовке и ссылке статьи.

20. `articles.append(row)`: Добавляем словарь `row` в список `articles`.

21. `return articles`: Возвращаем список `articles` из функции.

23. Пример использования функции:
    - Создаем список запросов `queries`, содержащий ключевые слова или фразы для поиска статей на Habr.
    - Вызываем функцию `scrape_habr_articles` с передачей списка запросов `queries`.
    - Результаты сохраняются в переменной `articles`.

24. `for article in articles:`: Начинаем цикл `for`, который перебирает каждую статью `article` из списка `articles`.

25. `print(article)`: Выводим информацию о каждой статье на экран.


## Дополнительная часть (необязательная)

Вам необходимо написать функцию, которая будет основана на **поиске** по сайту [habr.com](https://habr.com/ru/search/).
Функция в качестве параметра должна принимать **список** запросов для поиска (например, `['python', 'анализ данных']`) и на основе материалов, попавших в результаты поиска по **каждому** запросу, возвращать датафрейм вида:

```
<дата> - <заголовок> - <ссылка на материал>
```

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.


Функция из обязательной части задания должна быть расширена следующим образом:
- кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента `4` необходимо получить материалы с первых 4 страниц результатов;
- в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
```
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>
```
